In [1]:
# Dependencies
import json
import matplotlib.pyplot as plt
import requests
from pprint import pprint
from scipy import stats
import pandas as pd
from sodapy import Socrata

import numpy as np
from IPython.display import display
import datetime

In [2]:
# Specify URL
client = Socrata("data.cdc.gov", None)

# Make & store response
response = client.get("n8mc-b4w4", limit=1000000)

In [23]:
# Make DataFrame
response_df = pd.DataFrame.from_records(response)
response_df.head(40)

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_positive_specimen,underlying_conditions_yn
0,2021-02,MD,24,MONTGOMERY,24031,18 to 49 years,Male,White,Non-Hispanic/Latino,0,Missing,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,No,NaN,NaN
1,2022-01,CA,06,SAN JOAQUIN,06077,18 to 49 years,Male,NA,NA,NaN,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN
2,2021-02,MA,25,MIDDLESEX,25017,50 to 64 years,Male,NA,NA,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,No,0,NaN
3,2022-06,PA,42,PHILADELPHIA,42101,0 - 17 years,Male,Unknown,Hispanic/Latino,NaN,Missing,Missing,Probable Case,Unknown,Unknown,Unknown,Unknown,0,NaN
4,2021-01,TN,47,DAVIDSON,47037,50 to 64 years,Male,White,Non-Hispanic/Latino,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,0,NaN
5,2020-04,CA,06,SACRAMENTO,06067,65+ years,Male,White,Non-Hispanic/Latino,0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Yes,No,Missing,NaN,NaN
6,2020-12,CA,06,FRESNO,06019,18 to 49 years,Female,Unknown,Unknown,NaN,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN
7,2021-08,NJ,34,MIDDLESEX,34023,65+ years,Male,White,Non-Hispanic/Latino,0,Missing,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,No,1,NaN
8,2021-01,MA,25,ESSEX,25009,18 to 49 years,Female,NA,NA,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,0,NaN
9,2020-12,CA,06,ORANGE,06059,50 to 64 years,Male,Unknown,Unknown,NaN,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN


In [45]:
# Create variable
case_count_by_state = response_df["res_state"].groupby(response_df["res_state"]).count()


In [5]:
# Sanity check
total_cases = case_count_by_state.sum()
total_cases

1000000

In [6]:
# Total unique states
total_states = case_count_by_state.nunique()
total_states

22

In [19]:
# Create Df showing cases in each state
cases_count_by_state_df = response_df[['res_state']]

cases_count_by_state_df['case_count'] = 0
cases_count_by_state_df = cases_count_by_state_df.groupby(['res_state'])['case_count'].agg('count').reset_index()
cases_count_by_state_df

C:\Users\cnr2231\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,res_state,case_count
0,AL,8893
1,AZ,24138
2,CA,162339
3,CO,26666
4,FL,74382
5,GA,24137
6,IL,42215
7,IN,12068
8,MA,52630
9,MD,36205


In [8]:
# Create variable to hold cases by month per state

cases_count_by_state_month_df = response_df[['res_state', 'case_month']]

cases_count_by_state_month_df['case_count'] = 0
cases_count_by_state_month_df = cases_count_by_state_month_df.groupby(['res_state','case_month'])['case_count'].agg('count').reset_index()
cases_count_by_state_month_df.head()

C:\Users\cnr2231\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,res_state,case_month,case_count
0,AL,2020-03,29
1,AL,2020-04,28
2,AL,2020-06,723
3,AL,2020-07,138
4,AL,2020-08,189


In [9]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [10]:
abr_df = pd.DataFrame(abbrev_to_us_state.items(), columns=["res_state", "jurisdiction"])
abr_df.head()

,res_state,jurisdiction
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [11]:
cases_df = cases_count_by_state_df.merge(abr_df, how="inner")
cases_df.head()

#export

,res_state,case_count,jurisdiction
0,AL,8893,Alabama
1,AZ,24138,Arizona
2,CA,162339,California
3,CO,26666,Colorado
4,FL,74382,Florida


In [12]:
#read moderna and pfizer data
moderna_df = pd.read_csv("Resources/cdc-moderna-covid-19-vaccine-distribution-by-state.csv")
pfizer_df = pd.read_csv("Resources/cdc-pfizer-covid-19-vaccine-distribution-by-state.csv")

#renaming identical column names
moderna_df = moderna_df.rename(columns={'_1st_dose_allocations': '1st_dose_Moderna',"_2nd_dose_allocations" :"2nd_dose_Moderna"})
#display(moderna_df)

pfizer_df = pfizer_df.rename(columns={'_1st_dose_allocations': '1st_dose_Pfizer',"_2nd_dose_allocations" :"2nd_dose_Pfizer"})
#display(pfizer_df)


vaccines_merge_df = moderna_df.merge(pfizer_df, how="left")
#display(vaccines_merge_df)

#have a proper date format for "week of allocations" columns
vaccines_merge_df[['Week_of_Allocations','Time']] = vaccines_merge_df.week_of_allocations.str.split("T",expand=True,)
vaccines_merge_df = vaccines_merge_df[["jurisdiction","1st_dose_Moderna","2nd_dose_Moderna","1st_dose_Pfizer","2nd_dose_Pfizer","Week_of_Allocations"]]

#export merged data to csv for business use
vaccines_merge_df.to_excel ('Output_Files/vaccines_merge_df.xlsx',header=True)

display(vaccines_merge_df)

,jurisdiction,1st_dose_Moderna,2nd_dose_Moderna,1st_dose_Pfizer,2nd_dose_Pfizer,Week_of_Allocations
0,Connecticut,41220,41220,54360,54360,2021-06-21
1,Maine,15800,15800,21420,21420,2021-06-21
2,Massachusetts,79500,79500,104580,104580,2021-06-21
3,New Hampshire,15800,15800,21420,21420,2021-06-21
4,Rhode Island,12480,12480,17280,17280,2021-06-21
...,...,...,...,...,...,...
995,Hawaii,14200,14200,18720,18720,2021-03-08
996,Nevada,28500,28500,36270,36270,2021-03-08
997,American Samoa,0,0,0,0,2021-03-08
998,Guam,0,0,0,0,2021-03-08


In [36]:
total_jurisdictions = vaccines_merge_df.groupby(['jurisdiction'])['1st_dose_Moderna', '2nd_dose_Moderna','1st_dose_Pfizer','2nd_dose_Pfizer'].agg('sum').reset_index()
display(total_jurisdictions)


C:\Users\cnr2231\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,jurisdiction,1st_dose_Moderna,2nd_dose_Moderna,1st_dose_Pfizer,2nd_dose_Pfizer
0,Alabama,834960,834960,1131930,1131930
1,Alaska,151860,151860,226440,214740
2,American Samoa,5000,0,11700,0
3,Arizona,1183560,1183560,1603440,1603440
4,Arkansas,506920,506920,691470,691470
...,...,...,...,...,...
58,Virginia,1450280,1450280,1966500,1966500
59,Washington,1196520,1196520,1621890,1621890
60,West Virginia,323960,323960,442800,442800
61,Wisconsin,1000800,1000800,1350090,1350090


In [44]:
#merge based on jurisdiction/state
cases_and_vaccinations_df = cases_df.merge(total_jurisdictions, how="inner")
display(cases_and_vaccinations_df)

#export
cases_and_vaccinations_df.to_excel('Output_Files/Cases_vs_Vaccinations_df.xlsx',header=True)

,res_state,case_count,jurisdiction,1st_dose_Moderna,2nd_dose_Moderna,1st_dose_Pfizer,2nd_dose_Pfizer
0,AL,8893,Alabama,834960,834960,1131930,1131930
1,AZ,24138,Arizona,1183560,1183560,1603440,1603440
2,CA,162339,California,6658300,6658300,8980920,8980920
3,CO,26666,Colorado,946800,946800,1283220,1283220
4,FL,74382,Florida,3642160,3642160,4910130,4910130
5,GA,24137,Georgia,1726760,1726760,2336580,2336580
6,IL,42215,Illinois,1728160,1728160,2336580,2336580
7,IN,12068,Indiana,1122040,1122040,1517850,1517850
8,MA,52630,Massachusetts,1208000,1208000,1637460,1637460
9,MD,36205,Maryland,1033200,1033200,1401120,1401120
